In [21]:
import glob
import os
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from werkzeug.utils import secure_filename
import cv2
import numpy as np
import cvlib as cv

def load_and_resize_image(face_img):
    p = Image.fromarray(face_img)
    r = p.resize((128, 128))
    return np.array(r)

def normalize_image(image_array):
    n = image_array / 255.0
    return (n ** 1.5) * 255.0

def clip_image_values(image_array):
    return np.clip(image_array, 0, 255).astype(np.uint8)

def convert_to_gray(image_array):
    return cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

def denoise_image(gray_image):
    return cv2.GaussianBlur(gray_image, (5, 5), 0)

def process_image(face_img):
    r = load_and_resize_image(face_img)
    n = normalize_image(r)
    c = clip_image_values(n)
    g = convert_to_gray(c)
    d = denoise_image(g)
    return d

for img in glob.glob(r"D:\projects\Main projects gec\face recognition\face recognition data/*"):
    for i in glob.glob(r"" + str(img) + "/*"):
        f = cv2.imread(i)
        faces, confidences = cv.detect_face(f)
        os.makedirs(r"D:\projects\Main projects gec\face recognition\Newdata2"+str(img[68:]), exist_ok=True)
        basepath = os.path.dirname(r"D:\projects\Main projects gec\face recognition\Newdata2"+str(img[68:]))
        for idx, face in enumerate(faces):
            (startX, startY, endX, endY) = face
            face_img = f[startY:endY, startX:endX]
            file_path = os.path.join(
                basepath, str(img[69:]), secure_filename(i))
            if face_img is not None and face_img.size > 0:
                processed_image = process_image(face_img)
                cv2.imwrite(file_path, cv2.cvtColor(processed_image, cv2.COLOR_GRAY2RGB))
            else:
                print(f"Skipping writing for {file_path} due to empty or None face_img.")


Skipping writing for D:\projects\Main projects gec\face recognition\Newdata\Ram Charan\D_projects_Main_projects_gec_face_recognition_face_recognition_data_Ram_Charan_10.jpg due to empty or None face_img.
Skipping writing for D:\projects\Main projects gec\face recognition\Newdata\Ram Charan\D_projects_Main_projects_gec_face_recognition_face_recognition_data_Ram_Charan_10.jpg due to empty or None face_img.
Skipping writing for D:\projects\Main projects gec\face recognition\Newdata\Ram Charan\D_projects_Main_projects_gec_face_recognition_face_recognition_data_Ram_Charan_10.jpg due to empty or None face_img.
Skipping writing for D:\projects\Main projects gec\face recognition\Newdata\Ram Charan\D_projects_Main_projects_gec_face_recognition_face_recognition_data_Ram_Charan_10.jpg due to empty or None face_img.
Skipping writing for D:\projects\Main projects gec\face recognition\Newdata\Ram Charan\D_projects_Main_projects_gec_face_recognition_face_recognition_data_Ram_Charan_8.jpg due to empty

In [20]:
import cv2
import numpy as np
import os
import random

def rotate_image(image, angle):
    r, c, _ = image.shape
    re = cv2.getRotationMatrix2D((c / 2, r / 2), angle, 1)
    ro = cv2.warpAffine(image, re, (c, r))
    return ro

def flip_image_horizontal(image):
    f = cv2.flip(image, 1)
    return f

def adjust_brightness(image, factor):
    h = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h[:,:,2] = np.clip(h[:,:,2] * factor, 0, 255)
    b = cv2.cvtColor(h, cv2.COLOR_HSV2BGR)
    return b

def random_crop(image, crop_size):
    r, c, _ = image.shape
    x = np.random.randint(0, c - crop_size[1] + 1)
    y = np.random.randint(0, r - crop_size[0] + 1)
    c = image[y:y+crop_size[0], x:x+crop_size[1]]
    return cr

def random_scale(image, scale_range):
    s = np.random.uniform(scale_range[0], scale_range[1])
    sc = cv2.resize(image, None, fx=s, fy=s)
    return sc

def add_noise(image, noise_level):
    n = np.random.normal(0, noise_level, image.shape)
    no = np.clip(image + noise, 0, 255)
    return no

def augment_and_save_images(input_folder, output_folder, max_output_images=50):
    os.makedirs(output_folder, exist_ok=True)
    fe = os.listdir(input_folder)
    random.shuffle(fe)
    g = 0
    for f in fe:
        if g >= max_output_images:
            break
        im = os.path.join(input_folder, f)
        image = cv2.imread(im)
        for i in range(2):
            au = random.choice(['rotate', 'flip', 'brightness'])
            if au == 'rotate':
                rotated_img = rotate_image(image, np.random.uniform(-30, 30))
                augmented_img = rotated_img
            elif au == 'flip':
                if np.random.rand() > 0.5:
                    flipped_img = flip_image_horizontal(image)
                    augmented_img = flipped_img
                else:
                    augmented_img = image
            elif au == 'brightness':
                brightness_factor = np.random.uniform(0.5, 1.5)
                brightened_img = adjust_brightness(image, brightness_factor)
                augmented_img = brightened_img
            output_filename = f"aug_{g}_{au}_{f}"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, augmented_img)
            g += 1



In [21]:
import glob
for img in glob.glob(r"D:\projects\Main projects gec\face recognition\Newdata/*"):
    input_folder = img
    output_folder =img 
    augment_and_save_images(input_folder, output_folder)


In [22]:
import os
import shutil

def move_folder_to_new_directory(old_folder_path, new_folder_name):
    new_folder_path = os.path.join(os.path.dirname(old_folder_path), new_folder_name)
    os.makedirs(new_folder_path, exist_ok=True)
    shutil.move(old_folder_path, new_folder_path)
    print(f"Folder '{os.path.basename(old_folder_path)}' moved to '{new_folder_path}'")
for i in glob.glob(r"D:\projects\Main projects gec\face recognition\Newdata/*"):
    new_folder_name = "x"+i[55:]
    move_folder_to_new_directory(i, new_folder_name)
    

In [26]:
import os
import glob
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = [224, 224]

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

for i in glob.glob(r"D:\projects\Main projects gec\face recognition\Newdata/*"):
    training_set = train_datagen.flow_from_directory(
        i,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )
    
    test_set = test_datagen.flow_from_directory(
        i,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )
    
    try:
        r = model.fit_generator(
            training_set,
            validation_data=test_set,
            epochs=10,
            steps_per_epoch=len(training_set),
            validation_steps=len(test_set)
        )
    except FileNotFoundError as e:
        print(f"Error: {e}")
        continue  
    
    name = i[55:]
    model_path = os.path.join(
        r"D:\projects\Main projects gec\face recognition\models", f'{name}.h5')
    model.save(model_path)


Found 153 images belonging to 1 classes.
Found 153 images belonging to 1 classes.


C:\Users\amang\AppData\Local\Temp\ipykernel_17308\2322451824.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
5/5 [==============================] - 33s 8s/step - loss: 8.6980e-10 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 45s 10s/step - loss: 1.0678e-09 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 44s 10s/step - loss: 1.2857e-09 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 45s 10s/step - loss: 1.2060e-09 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 48s 11s/step - loss: 1.1123e-09 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 46s 10s/step - loss: 1.2433e-09 - accuracy: 1.0000 - val_loss: 1.0451e-09 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 47s 10s/step - loss: 1.2036e-09 - accuracy: 1.0000 - val_loss: 1.0451

Epoch 4/10
3/3 [==============================] - 29s 11s/step - loss: 4.5304e-09 - accuracy: 1.0000 - val_loss: 4.4292e-09 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 29s 11s/step - loss: 4.3390e-09 - accuracy: 1.0000 - val_loss: 4.4291e-09 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 29s 11s/step - loss: 3.4168e-09 - accuracy: 1.0000 - val_loss: 4.4291e-09 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 29s 11s/step - loss: 3.8708e-09 - accuracy: 1.0000 - val_loss: 4.4291e-09 - val_accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 29s 13s/step - loss: 5.2061e-09 - accuracy: 1.0000 - val_loss: 4.4291e-09 - val_accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 29s 11s/step - loss: 3.8883e-09 - accuracy: 1.0000 - val_loss: 4.4291e-09 - val_accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 29s 11s/step - loss: 4.9727e-09 - accuracy: 1.0000 - val_loss: 4.42

In [88]:
import sqlite3

try:
    # Connect to the SQLite database (or create a new one if it doesn't exist)
    conn = sqlite3.connect(r'verify_data.db')
    cursor = conn.cursor()

    # Create a table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY,
            pickle_data BLOB,
            name TEXT,
            username TEXT,
            password TEXT,
            details TEXT
        )
    ''')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    print("SQLite error:", e)

finally:
    # Close the connection
    conn.close()


In [88]:
p="D:\projects\Main projects gec\face recognition\models"
len(p)

52

In [23]:
import sqlite3
for i in glob.glob(r"D:\projects\Main projects gec\face recognition\models/*"):
    conn = sqlite3.connect('verify_data.db')
    cursor = conn.cursor()
    with open(i, 'rb') as file:
        h5_data = file.read()
    cursor.execute('''
        INSERT INTO users (pickle_data, name, username, password, details)
        VALUES (?, ?, ?, ?, ?)
    ''', (sqlite3.Binary(h5_data),i[53:],i[53:]+"123", 'password123', 'Additional details'))

    conn.commit()
    conn.close()


In [111]:
for i in glob.glob(r"D:\projects\Main projects gec\face recognition\Newdata\xSamantha\Samantha/*"):
    face_crop = cv2.imread(i)
    face_crop = cv2.resize(face_crop, (224,224))
    face_crop = face_crop.astype("float") / 255.0
    face_crop = img_to_array(face_crop)
    face_crop = np.expand_dims(face_crop, axis=0)
    p = model.predict(face_crop)
    print(p)
    

1/1 [==============================] - 0s 170ms/step
[[1.]]
1/1 [==============================] - 0s 155ms/step
[[1.]]
1/1 [==============================] - 0s 155ms/step
[[1.]]
1/1 [==============================] - 0s 164ms/step
[[1.]]
1/1 [==============================] - 0s 155ms/step
[[1.]]
1/1 [==============================] - 0s 154ms/step
[[1.]]
1/1 [==============================] - 0s 149ms/step
[[1.]]
1/1 [==============================] - 0s 169ms/step
[[1.]]
1/1 [==============================] - 0s 154ms/step
[[1.]]
1/1 [==============================] - 0s 186ms/step
[[1.]]
1/1 [==============================] - 0s 233ms/step
[[1.]]
1/1 [==============================] - 0s 283ms/step
[[1.]]
1/1 [==============================] - 0s 278ms/step
[[1.]]
1/1 [==============================] - 0s 294ms/step
[[1.]]
1/1 [==============================] - 0s 282ms/step
[[1.]]
1/1 [==============================] - 0s 288ms/step
[[1.]]
1/1 [==============================] - 0